In [1]:
import numpy as np
import pickle
import os.path
import tensorflow as tf
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

# Read local data

In [2]:
def read_local(file):
    with open(file, 'rb') as file_bytes:
        return pickle.load(file_bytes)

# SVM Section

In [32]:
probs, indicies, labels = read_local('svm_probabilities.pickle'), read_local('svm_indicies.pickle'), read_local('svm_labels.pickle')
probs, labels = np.asarray(probs), np.asarray(labels)
z = np.vstack(probs)
l = np.concatenate(labels).reshape((-1,1))
Y = label_binarize(l, classes= [0,1,2])
n_classes = Y.shape[1]
fpr, tpr, roc_auc = dict(), dict(), dict() 
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y[:, i], z[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Interpolate
from scipy import interp
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Average and AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

import matplotlib.pyplot as plt
from itertools import cycle
plt.figure()

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
classes = ['Normal', 'MCI', 'AD']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('SVM_Roc', dpi=300)

# Deep learn section

In [9]:
# data = read_local('valid_predictions.pickle')
# labels = read_local('true_labels.pickle')
data = read_local('dl_validation.pickle')
labels = read_local('dl_validation_labels.pickle')
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
pred = tf.placeholder(tf.float32, name='pred')
p_original = tf.argmax(pred)
softmax = tf.nn.softmax(pred)
p_softmax = tf.argmax(softmax)

In [10]:
softmax_probabilities = session.run(softmax, feed_dict={pred: data.T})
max_prob = np.amax(softmax_probabilities,axis=1).reshape((-1,1))
clean_class = session.run(p_original, feed_dict={pred: data}).reshape((-1,1))
count = 0
for row in softmax_probabilities:    
    for col in row:
        print('|{0:.5f}'.format(col), end='')    
    print('| -> {0:.5f} -> {1:0.5f}'.format(max_prob[count][0], clean_class[count][0]))
    count = count + 1

|0.65476|0.00456|0.34068| -> 0.65476 -> 0.00000
|1.00000|0.00000|0.00000| -> 1.00000 -> 0.00000
|0.91528|0.07309|0.01163| -> 0.91528 -> 0.00000
|0.99883|0.00114|0.00003| -> 0.99883 -> 0.00000
|0.00038|0.00044|0.99918| -> 0.99918 -> 2.00000
|0.87537|0.00893|0.11570| -> 0.87537 -> 0.00000
|0.00138|0.00051|0.99811| -> 0.99811 -> 2.00000
|0.99734|0.00046|0.00220| -> 0.99734 -> 0.00000
|0.00001|0.00000|0.99999| -> 0.99999 -> 2.00000
|1.00000|0.00000|0.00000| -> 1.00000 -> 0.00000
|0.00000|0.00000|1.00000| -> 1.00000 -> 2.00000
|0.00040|0.00187|0.99774| -> 0.99774 -> 2.00000
|0.37832|0.62019|0.00149| -> 0.62019 -> 1.00000
|0.27432|0.00118|0.72450| -> 0.72450 -> 2.00000
|0.00002|0.00003|0.99996| -> 0.99996 -> 2.00000
|0.99558|0.00090|0.00352| -> 0.99558 -> 0.00000
|0.00002|0.00195|0.99803| -> 0.99803 -> 2.00000
|0.99589|0.00358|0.00053| -> 0.99589 -> 0.00000
|0.99970|0.00020|0.00010| -> 0.99970 -> 0.00000
|0.99826|0.00041|0.00133| -> 0.99826 -> 0.00000
|0.97831|0.00043|0.02126| -> 0.97831 -> 

In [12]:
Y_hot = label_binarize(labels, classes= [0,1,2])

# ROC and AUC for devset

In [13]:
n_classes = Y_hot.shape[1]
fpr, tpr, roc_auc = dict(), dict(), dict() 
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_hot[:, i], softmax_probabilities[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [17]:
# Aggrefate all false positives rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Interpolate
from scipy import interp
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Average and AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

import matplotlib.pyplot as plt
from itertools import cycle
plt.figure()

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
classes = ['Normal', 'MCI', 'AD']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.savefig('DL_ROC', dpi=300)